In [14]:
import pymysql
import pandas as pd

def query_db(host,query, params):
    conn = pymysql.connect(host =host, read_default_file="/etc/mysql/conf.d/analytics-research-client.cnf")
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute(query, params)
    rows = cursor.fetchall()
    conn.close()
    return mysql_to_pandas(rows)


def mysql_to_pandas(dicts):
    dmaster = {}
    for d in dicts:
        for k in d.keys():
            if k not in dmaster:
                dmaster[k] = []
            
            dmaster[k].append(d[k]) 
    return pd.DataFrame(dmaster)

def decode(x):
    try:
         return x.decode('utf-8')
    except:
        return x


In [15]:
cx = query_db( 'x1-slave.eqiad.wmnet', "SELECT * FROM wikishared.cx_translations WHERE translation_started_by = translation_last_update_by  ", {})

In [42]:
import json
import time

def get_uname(uid):
    query = "SELECT gu_name from centralauth.globaluser where gu_id = %(id)s"
    try:
        return query_db( 'analytics-store.eqiad.wmnet', query,  {'id': uid  })['gu_name'][0]
    except:
        return None

t1 = time.time()
cx['user'] =  cx['translation_last_update_by'].apply(lambda x: get_uname(int(x)))
t2 = time.time()
print t2 - t1

 425.657828093


In [43]:
cx.head()

,translation_id,translation_last_update_by,translation_last_updated_timestamp,translation_progress,translation_source_language,translation_source_title,translation_source_url,translation_start_timestamp,translation_started_by,translation_status,translation_target_language,translation_target_title,translation_target_url,user
0,52,33245755,20150314062654,"{""any"":0.09470053308247099,""human"":0.032013455...",sv,Frankrike,//sv.wikipedia.org/wiki/Frankrike,20150121071556,33245755,draft,da,Frankrike,None,Erik Moeller (WMF)
1,53,33245755,20150116013359,"{""any"":0.04598201334493763,""human"":0,""mt"":0.04...",ca,Fre,//ca.wikipedia.org/wiki/Fre,20150116013359,33245755,draft,nb,Fre,None,Erik Moeller (WMF)
2,57,8907984,20150116032056,"{""any"":0.9744360902255639,""human"":0,""mt"":0.974...",es,Palak paneer,//es.wikipedia.org/wiki/Palak paneer,20150116032056,8907984,published,pt,Palak paneer,//pt.wikipedia.org/wiki/User:Santhosh.thotting...,Santhosh.thottingal
3,115,31180681,20150116081419,"{""any"":0.11593383288562138,""human"":0,""mt"":0.11...",en,BuzzFeed,//en.wikipedia.org/wiki/BuzzFeed,20150116081419,31180681,draft,pt,BuzzFeed,None,Fábio B. Lima
4,116,29633141,20150116090346,"{""any"":0.6402660217654171,""human"":0.5269044740...",es,Té de burbujas,//es.wikipedia.org/wiki/Té de burbujas,20150116090346,29633141,deleted,ca,Usuari:읏/Te de bombolles,None,읏


In [54]:
rec_file = "/home/ellery/en-fr/staff-personal-recs-sent.json"
f = open(rec_file)
recs = json.load(f)
tuples = []
for obj in recs:
    user = obj['user']
    for rec in obj['recommendations']:
        title = rec['title'].replace('_', ' ')
        #score = rec['score']
        tuples.append((user, title))
d = pd.DataFrame(tuples)
d.columns = ['user', 'translation_source_title']
        


In [56]:
df = d.merge(cx, how = 'left', on = ['user', 'translation_source_title'])

/home/ellery/anaconda/lib/python2.7/site-packages/pandas/tools/merge.py:623: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  rlab = rizer.factorize(rk)


In [59]:
df

,user,translation_source_title,translation_id,translation_last_update_by,translation_last_updated_timestamp,translation_progress,translation_source_language,translation_source_url,translation_start_timestamp,translation_started_by,translation_status,translation_target_language,translation_target_title,translation_target_url
0,Guillom,Louise Griffiths,44133,29,20150617160602,"{""any"":0.022959763582632416,""human"":0.02295976...",en,//en.wikipedia.org/wiki/Louise Griffiths,20150617160602,29,draft,fr,Louise Griffiths,None
1,Guillom,Kendra Jade Rossi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Guillom,Jake Weary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Guillom,Todd Glass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Guillom,Ashley Argota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Kouiskas,Prescription Drug User Fee Act,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Kouiskas,Demand reduction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Kouiskas,IPLEDGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Kouiskas,Economic impact of HIV/AIDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Kouiskas,Expanded access,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
